In [1]:
# import libs
import pandas as pd
import glob
import os
import re
%matplotlib inline

In [2]:
# define global variables
elec_folder_path = '/users/ianmyjer/desktop/disagg/elec/'
pre_combined_elec_path = '/users/ianmyjer/desktop/disagg/pre_combined/Electricity_P.tab'
weather_hourly_path = '/users/ianmyjer/desktop/disagg/weather/Climate_HourlyWeather.tab'

In [63]:
# Define function to import all files from folder and combine
# based on datetime index (axis = 1)
def tsvs_from_folder(folder_path,meter_type='Electricity'):
    # get all filepaths in folder
    allfiles = glob.glob(folder_path+"/*.tab")
    
    # create a list with the name of each submeter
    name = [re.search('{}_(.*).tab$'.format(meter_type),fp).group(1) for fp in allfiles]

    # provided the file name and corresponding path to each file
    # read in file and parse datetime
    # store in dictionary of dataframes
    datadct = {}
    for k, v in zip(name,allfiles):
        print(k)
        datadct[k] = pd.read_csv(v,delimiter='\t',header=0)
        datadct[k]['ts'] = pd.to_datetime(datadct[k]['unix_ts'],unit='s')
        datadct[k].set_index('ts',inplace=True)
        datadct[k].drop('unix_ts',axis=1,inplace=True)

    # join everything together based on index        
    df = pd.concat(datadct,axis=1,join='outer')
    df.index.rename('ts',inplace=True)

    return df

In [64]:
# import elec files using function
elec = tsvs_from_folder(elec_folder_path)

B1E
B2E
BME
CDE
CWE
DNE
DWE
EBE
EQE
FGE
FRE
GRE
HPE
HTE
OFE
OUE
RSE
TVE
UTE
WHE
WOE


In [5]:
# take only the "power" columns
df = elec.xs('P',level=1,axis=1).copy()

In [6]:
# add extra columns based on documentation
df['MHE'] = df['WHE'] - df['RSE'] - df['GRE']
df['UNE'] = df['MHE'] - df.drop(['MHE','WHE','RSE','GRE'],axis=1).sum(axis=1)

In [7]:
# Rename columns
actual_names = {'B1E':'north_br',
                'B2E':'south_br',
                'BME':'basement',
                'CDE':'dryer',
                'CWE':'washer',
                'DNE':'dining_room',
                'DWE':'dishwasher',
                'EBE':'workbench',
                'EQE':'security_system',
                'FGE':'refrigerator',
                'FRE':'furnace_fan',
                'GRE':'garage',
                'HPE':'heat_pump',
                'HTE':'dhw_heater',
                'MHE':'main_house_total',
                'OFE':'office',
                'OUE':'outside_plug',
                'RSE':'rental_suite',
                'TVE':'entertainment',
                'UTE':'utility_room',
                'UNE':'unmetered',
                'WHE':'whole_house_total',
                'WOE':'oven'}
df.rename(columns = actual_names,inplace=True)

In [8]:
# import weather data
weather = pd.read_csv(weather_hourly_path, 
                      index_col=0, 
                      delimiter='\t', 
                      header=0, 
                      parse_dates=True,
                      usecols=['Date/Time','Temp (C)',
                                'Dew Point Temp (C)',
                                'Rel Hum (%)',
                                'Wind Spd (km/h)',
                                'Stn Press (kPa)'])
weather.index.name = 'ts'

In [9]:
weather.isnull().sum()

Temp (C)              22
Dew Point Temp (C)    36
Rel Hum (%)           38
Wind Spd (km/h)       31
Stn Press (kPa)       35
dtype: int64

In [10]:
# Fill N/A values using pandas interpolate
weather.interpolate(method='time',axis=0,inplace=True)

In [11]:
weather.isnull().sum()

Temp (C)              0
Dew Point Temp (C)    0
Rel Hum (%)           0
Wind Spd (km/h)       0
Stn Press (kPa)       0
dtype: int64

In [12]:
# super stupid code
# but somehow works pretty fast
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour
weather['Year'] = weather.index.year
weather['Month'] = weather.index.month
weather['Day'] = weather.index.day
weather['Hour'] = weather.index.hour
ymdh = ['Year','Month','Day','Hour']
merged = pd.merge(df,weather,on=ymdh,how='left').drop(ymdh,axis=1)
merged.index = df.index
merged.dropna(axis=0,inplace=True)

In [14]:
merged.to_csv('/users/ianmyjer/desktop/disagg/electric_data_with_weather.csv')